NLP PROJECT

In [1]:
#Set up the environment
%pip install -r ../requirements.txt

#Use only if you are using Google Colab
#from google.colab import files
#uploaded = files.upload()


Note: you may need to restart the kernel to use updated packages.


In [3]:
!#pip install transformers torch

import pandas as pd
import nltk
from transformers import pipeline

df = pd.read_csv('../data/proverbs.csv')
print(df.head())


                               Proverb Language  \
0                  Δώσε τόπο στην οργή    Greek   
1  Αγάλι-αγάλι γίνεται η αγουρίδα μέλι    Greek   
2   Είπε ο γάιδαρος τον πετεινό κεφάλα    Greek   
3     Η καλύτερη άμυνα είναι η επίθεση    Greek   
4              Η φτήνια τρώει τον παρά    Greek   

                        Literal English Translation  \
0                               Give place to anger   
1  Slowly, slowly the unripe fruit turns into honey   
2           The donkey called the rooster bigheaded   
3                The best defence is a good offense   
4                          Cheapness eats the money   

                                             Meaning Irony (Yes/No)  
0  Control or suppress your anger and avoid actin...            Yes  
1  Good things take time, and with patience, some...             No  
2  Someone is criticizing another for a flaw that...            Yes  
3  Taking proactive, aggressive action is often t...            Yes  
4  Cheap opt

In [4]:

# Drop rows with missing values in the specified columns
df = df.dropna(subset=['Literal English Translation', 'Meaning'])

from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
    return ' '.join(tokens)

# Apply preprocessing
df['Literal English Translation'] = df['Literal English Translation'].apply(preprocess)
df['Meaning'] = df['Meaning'].apply(preprocess)




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/giorgosdaskalopoulos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/giorgosdaskalopoulos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/giorgosdaskalopoulos/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
# Initialize sentiment analysis pipeline using a pre-trained transformer model
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

def get_sentiment(text):
    result = sentiment_pipeline(text)[0]
    # Convert the label to a sentiment score similar to compound score
    score = result['score'] if result['label'] == 'POSITIVE' else -result['score']
    return score

# Apply sentiment analysis using the transformer model
df['literal_sentiment'] = df['Literal English Translation'].apply(lambda x: get_sentiment(x))
df['meaning_sentiment'] = df['Meaning'].apply(lambda x: get_sentiment(x))

print(df[['Literal English Translation', 'literal_sentiment', 'Meaning', 'meaning_sentiment']])



               Literal English Translation  literal_sentiment  \
0                         give place anger          -0.984260   
1   slowly slowly unripe fruit turns honey          -0.592939   
2          donkey called rooster bigheaded          -0.981749   
3                best defence good offense           0.999724   
4                     cheapness eats money          -0.998331   
..                                     ...                ...   
91                                   green           0.993437   
92                              wash hands          -0.988124   
93                             put dots 's           0.853201   
94                     either works breaks          -0.817359   
95                  ’ discovered hot water           0.990440   

                                              Meaning  meaning_sentiment  
0     control suppress anger avoid acting impulsively           0.993691  
1   good things take time patience something seems...           0.992

In [6]:
threshold = 0.05

# Calculate the sentiment difference and classify irony
df['sentiment_diff'] = abs(df['literal_sentiment'] - df['meaning_sentiment'])
#df['sentiment_diff'] = abs(abs(df['literal_sentiment']) - abs(df['meaning_sentiment']))
df['predicted_irony'] = df['sentiment_diff'].apply(lambda x: 'Ironic' if x > threshold else 'Not Ironic')

print(df[['Literal English Translation', 'literal_sentiment', 'Meaning', 'meaning_sentiment', 'predicted_irony']].head())

df['Irony (Yes/No)'] = df['Irony (Yes/No)'].astype(str).str.lower().map({'yes': 1, 'no': 0})
#3 of the greek ones result in Nan for Irony, dont know why - just drop those 3
df = df.dropna(subset=['Irony (Yes/No)'])

# Convert 'predicted_irony' to numeric values
df['predicted_irony'] = df['predicted_irony'].map({'Ironic': 1, 'Not Ironic': 0})

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df['Irony (Yes/No)'], df['predicted_irony'])

print(f'Accuracy: {accuracy}')

              Literal English Translation  literal_sentiment  \
0                        give place anger          -0.984260   
1  slowly slowly unripe fruit turns honey          -0.592939   
2         donkey called rooster bigheaded          -0.981749   
3               best defence good offense           0.999724   
4                    cheapness eats money          -0.998331   

                                             Meaning  meaning_sentiment  \
0    control suppress anger avoid acting impulsively           0.993691   
1  good things take time patience something seems...           0.992571   
2           someone criticizing another flaw possess          -0.988330   
3  taking proactive aggressive action often best ...           0.882297   
4                 cheap options end costing long run          -0.999523   

  predicted_irony  
0          Ironic  
1          Ironic  
2      Not Ironic  
3          Ironic  
4      Not Ironic  
Accuracy: 0.5729166666666666


In [7]:
from sklearn.metrics import confusion_matrix

# Assuming 'Irony (Yes/No)' is the true label and 'predicted_irony' is the prediction
conf_matrix = confusion_matrix(df['Irony (Yes/No)'], df['predicted_irony'])
print(conf_matrix)


[[18 24]
 [17 37]]


In [8]:
tn, fp, fn, tp = conf_matrix.ravel()  # Extract values from confusion matrix
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

Specificity: 0.42857142857142855


In [9]:
from sklearn.metrics import classification_report, confusion_matrix

# Get the classification report as a dictionary
report = classification_report(df['Irony (Yes/No)'], df['predicted_irony'], output_dict=True)

# Calculate confusion matrix
conf_matrix = confusion_matrix(df['Irony (Yes/No)'], df['predicted_irony'])
tn, fp, fn, tp = conf_matrix.ravel()

# Extract metrics
precision = report['1']['precision']  # Precision for Ironic class
recall = report['1']['recall']  # Recall for Ironic class (Sensitivity)
specificity = tn / (tn + fp)  # Specificity (True Negative Rate)
accuracy = report['accuracy']  # Overall accuracy

# Print the metrics
# precision: Out of all the proverbs the model predicted as ironic, what proportion was actually ironic?
print(f"Precision: {precision:.4f}")
#recall/sensitivity: Out of all the proverbs that were actually ironic, what proportion did the model correctly identify as ironic?
print(f"Recall (Sensitivity): {recall:.4f}")
#specifity: Out of all the proverbs that were actually not ironic, what proportion did the model correctly identify as not ironic?
print(f"Specificity: {specificity:.4f}")
print(f"Accuracy: {accuracy:.4f}")

# Print the overall report for reference
print(classification_report(df['Irony (Yes/No)'], df['predicted_irony']))

Precision: 0.6066
Recall (Sensitivity): 0.6852
Specificity: 0.4286
Accuracy: 0.5729
              precision    recall  f1-score   support

           0       0.51      0.43      0.47        42
           1       0.61      0.69      0.64        54

    accuracy                           0.57        96
   macro avg       0.56      0.56      0.56        96
weighted avg       0.57      0.57      0.57        96

